## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/DLC-Jupyter-Notebooks
print(project_root)
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import psycopg2
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.142"
conn = psycopg2.connect(
    dbname="deeplabcut_db", user="postgres", 
    password="1234", host=host, port="5432")
cursor = conn.cursor()


## Fetch id list

In [ ]:
import pandas as pd

task = 'ToyLight'

saline_q = f"""
SELECT id FROM dlc_table
WHERE task = '{task}' AND health = 'saline' AND head_x_norm IS NOT NULL AND trial_length>600
ORDER BY id;
"""
df = pd.read_sql_query(saline_q, conn)
saline_id = df['id'].to_list()
# IDs to remove
bad_saline_id = [363, 369, 431]
# Filter them out
saline_id = [i for i in saline_id if i not in bad_saline_id]

ghrelin_q = f"""
SELECT id FROM dlc_table
WHERE task = '{task}' AND health = 'ghrelin' AND head_x_norm IS NOT NULL AND trial_length>600
ORDER BY id;
"""
df = pd.read_sql_query(ghrelin_q, conn)
ghrelin_id = df['id'].to_list()
# IDs to remove
bad_ghrelin_id = [368, 386, 401, 406, 426]
# Filter them out
ghrelin_id = [i for i in ghrelin_id if i not in bad_ghrelin_id]

print(f"{task} saline_id: {saline_id}\n")
print(f"{task} ghrelin_id: {ghrelin_id}\n")


## Calculate distance

In [ ]:
import importlib
import Python_scripts.Feature_functions.motion_features
import Python_scripts.Data_analysis.plot_groupwise_bar
import Python_scripts.Data_analysis.compare_distributions


importlib.reload(Python_scripts.Feature_functions.motion_features)
importlib.reload(Python_scripts.Data_analysis.plot_groupwise_bar)
importlib.reload(Python_scripts.Data_analysis.compare_distributions)

from Python_scripts.Feature_functions.motion_features import (
    compute_motion_features, batch_compute_motion_feature
)
from Python_scripts.Data_analysis.plot_groupwise_bar import plot_groupwise_bar
from Python_scripts.Data_analysis.compare_distributions import compare_distributions


In [ ]:
saline_d = batch_compute_motion_feature(conn, saline_id, feature='distance', time_limit=None)
ghrelin_d = batch_compute_motion_feature(conn, ghrelin_id, feature='distance', time_limit=None)


### Distribution of distance per frame

In [ ]:
import numpy as np

fig, ax = compare_distributions(
    np.concatenate(saline_d),
    np.concatenate(ghrelin_d),
    labels=['Saline', 'Ghrelin'],
    kind='cdf'
)

ax.set_xlim(0, 0.075)
ax.set_title(f"{task}_distance_distribution.pdf")
fig.savefig(f"{task}_distance_distribution.pdf", dpi=300, bbox_inches='tight')


### Batch call with different max_time

In [ ]:
import numpy as np
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

pdf_path = f'{task}_distance_all_time.pdf'
with PdfPages(pdf_path) as pdf:
    for max_time in range(100, 1300, 100):
        print(f"[INFO] Analyzing max_time = {max_time} seconds")
        
        saline_d = batch_compute_motion_feature(conn, saline_id, feature='distance', time_limit=max_time)
        ghrelin_d = batch_compute_motion_feature(conn, ghrelin_id, feature='distance', time_limit=max_time)

        # 1. Total distance per trial
        saline_totals = [d.sum() for d in saline_d]
        ghrelin_totals = [d.sum() for d in ghrelin_d]
        
        # 2. Create DataFrame for plotting
        df = pd.DataFrame({
            'group': ['Saline'] * len(saline_totals) + ['Ghrelin'] * len(ghrelin_totals),
            'total_distance': saline_totals + ghrelin_totals
        })
        # Plot
        fig, ax = plot_groupwise_bar(df, y='total_distance', ylabel='Distance')

        ax.set_title(f"{task} | max_time = {max_time}s")
        pdf.savefig(fig)
        plt.close(fig)  # Prevents inline display in notebooks

print(f"[DONE] All plots saved to: {pdf_path}")


### Comparison of total distance traveled

In [ ]:
import pandas as pd

# 1. Total distance per trial
saline_totals = [d.sum() for d in saline_d]
ghrelin_totals = [d.sum() for d in ghrelin_d]

# 2. Create DataFrame for plotting
df = pd.DataFrame({
    'group': ['Saline'] * len(saline_totals) + ['Ghrelin'] * len(ghrelin_totals),
    'total_distance': saline_totals + ghrelin_totals
})

# 3. Plot
fig, ax = plot_groupwise_bar(
    df,
    y='total_distance',
    title='Total Distance Traveled by Group',
    ylabel='Total Distance (a.u.)',
    plot_type='box',
    order=['Saline', 'Ghrelin'],
    show_stats=True
)


In [ ]:
import pandas as pd

q = """SELECT id, genotype, task, health, num_frames, frame_rate, trial_length, date 
FROM dlc_table WHERE genotype='white' ORDER BY health
"""
df_col = pd.read_sql_query(q, conn)
print(df_col)
